In [6]:
import numpy as np
from glob import glob
import nibabel as nib
from os.path import join as opj
from numpy import median, rint, nan

from nipype.interfaces import spm
from nipype.interfaces import fsl
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip
from nipype.pipeline.engine import Workflow, Node, MapNode

# MATLAB - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

#other study-specific variables
project_home = '/Users/catcamacho/Dropbox/Projects/TH_NAR_ASL/proc/dev'
subjects_dir = project_home
subjects_list = ['ko']
raw_dir = 'raw'
output_dir = 'proc'

#pre-proc variables
numslices = 38
TR = 4.844
asl_slice_order = range(1,numslices+1)
middle_slice = np.around(median(asl_slice_order), decimals=-1)
middle_slice = middle_slice.astype(int)

#Population specific variables for ASL
nex_asl = 3 #number of excitations from the 3D ASL scan parameters
inversion_efficiency = 0.8 #from GE
background_supp_eff = 0.75 #from GE
efficiency = inversion_efficiency * background_supp_eff 
T1_blood = 1.6 #T1 of blood in seconds(1.6s at 3T and 1.4s at 1.5T)
sat_time = 2 #in seconds, from GE
partition_coeff = 0.9 #whole brain average in ml/g
scaling_factor = 32 #scaling factor, can be taken from PW dicom header
postlabel_delay = 2.025 #1.525 #post label delay in seconds
labeling_time = 1.450 #labeling time in seconds
T1gm = 1.2 #estimated T1 of grey matter

In [11]:
# preprocflow nodes

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subjid']),
                  name="infosource")
infosource.iterables = [('subjid', subjects_list)]

# SelectFiles
templates = {'asl': subjects_dir + '/raw/{subjid}/asl.nii.gz',
             'pw': subjects_dir + '/raw/{subjid}/pw.nii.gz',
             'pd': subjects_dir + '/raw/{subjid}/pd.nii.gz'}
selectfiles = Node(SelectFiles(templates), name="selectfiles")

#gunzip
gunzip = Node(Gunzip(), name="gunzip")

# Datasink
datasink = Node(DataSink(), name="datasink")
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

#split ASL sequence into pw.nii and pd.nii
def split_asl_vols(asl, pw, pd):
	trim.inputs.in_file = asl
	trim.inputs.out_file = pw
	trim.inputs.end_index = 1
	trim.run()
	trim.inputs.out_file = pd
	trim.inputs.end_index = 2
	trim.inputs.begin_index = 1
	trim.run()

#quantify CBF from PW volume
def quantify_cbf(pw_nifti1,pd_nifti1,efficiency,T1_blood,partition_coeff,scaling_factor,postlabel_delay,labeling_time,TR,T1gm):
    # set variables
    pw_data = pw_nifti1.get_data()
    pd_data = pd_nifti1.get_data()
    #pd_data = pd_data.astype(float)
    #pd_nonzero_data = pd_data[pd_data==0]=nan
    conversion = 6000 #to convert values from mL/g/s to mL/100g/min
    
    cbf_numerator = (conversion*partition_coeff*pw_data)*(np.exp(postlabel_delay/T1_blood))
    cbf_denominator = (2*efficiency*T1_blood)*(pd_data*(1/(1-np.exp((-1*TR)/T1gm))))*(1-np.exp((-1*labeling_time)/T1_blood))
    cbf_data = cbf_numerator/cbf_denominator
    
    cbf_vol = nib.Nifti1Image(cbf_data, pw_nifti1.affine)
    nib.save(cbf_vol,project_home + '/raw/ko/cbf_vol.nii.gz')

In [12]:
pw_nifti1 = nib.load(project_home + '/raw/ko/pw.nii.gz')
pd_nifti1 = nib.load(project_home + '/raw/ko/pd.nii.gz')
quantify_cbf(pw_nifti1,pd_nifti1,efficiency,T1_blood,partition_coeff,scaling_factor,postlabel_delay,labeling_time,TR,T1gm)


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:44: RuntimeWarning: divide by zero encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:44: RuntimeWarning: invalid value encountered in divide


In [ ]:
aslprocflow = Workflow(name="aslprocflow")

aslprocflow.base_dir = opj(subjects_dir)
aslprocflow.connect([(infosource, selectfiles, [('subjid', 'subjid')]),
                     (selectfiles, gunzip, [('func', 'in_file')]),
                     (gunzip, slice_timing, [('out_file','in_file')]),
                     (slice_timing, datasink, [('slice_time_corrected_file', 'slice_timing')]),
                     (slice_timing, norm,[('slice_time_corrected_file','source')]),
                     (norm, datasink, [('normalized_source', 'norm')]),
                    ])
aslprocflow.write_graph(graph2use='flat')
aslprocflow.run('MultiProc', plugin_args={'n_procs': 2})